Normalização do modelo BCS não linear

In [15]:
import matplotlib.pyplot as plt
from matplotlib import cm
#import importlib
import numpy as np
#import subrotinas

#importlib.import_module('subrotinas')
%matplotlib tk
from param import *

#exec(compile(open('param.py', "rb").read(), 'param.py', 'exec')) #% Roda arquivo com parâmetros do modelo BCS
exec(compile(open('subrotinas.py', "rb").read(), 'subrotinas.py', 'exec')) #% Roda arquivo

### Variáveis de entrada
f $\in$ (35,65) Hz <br>
zc $\in$ (0,100)% <br>
pm $\in$ ($2\cdot 10^6$) <br>

### Variáveis de estado
pbh $\in$ ($10^5, 8.5 \cdot 10^6$) <br>
pwh $\in$ ($2 \cdot 10^6, 5.2\cdot10^6$) <br>
q $\in (10^{-2},3\cdot 10^{⁻2})$ <br>

### Saídas
pin $\in$ ($40, 60.5) \cdot 10^6$ <br>
H $\in$ ($350, 1000$) <br>

Função para retornar os valores de xc e x0 dado que<br>
$x_{min}<x<x_{max}$ <br>

Nova variável<br>

$xb=\frac{x-x0}{xc}$

Assim <br>

$x=xb\cdot xc+x0$

Fazendo com que a nova variável $0<xb<1$ <br>


In [16]:
pbi=np.arange(1e5,pr,1000)
qr=PI*(pr-pbi)
# fig=plt.figure()
# ax=fig.add_axes()
# ax.plot(pbi,qr)

In [17]:
# Calculo dos limites do HEAD e delta de press�o
def LimH(fL,qL):
    def funcaoH(fi,qm): #vazão em m3/s
        qi=qm/3600
        q0 = qi/Cq*(f0/fi);
        H0 = -1.2454e6*q0**2 + 7.4959e3*q0 + 9.5970e2;
        H = CH*H0*(fi/f0)**2; # Head
        return H

    qx = np.arange(fL[0],fL[1],1) # 35 a 65
    fx = np.arange(qL[0],qL[1],1)
    X,Y = np.meshgrid(fx, qx) # grid of point
    Z = funcaoH(X, Y) # evaluation of the function on the grid
    return[min([valor for linha in Z for valor in linha]),max([valor for linha in Z for valor in linha])]

In [18]:
LimH([35,75],[35,65])

[-175.58596610338176, 1040.2211136671065]

In [19]:
# Definimos duas variáveis para teste nas funções qch
# Make data.
# Calculo dos limites do HEAD e delta de press�o
pm=20e5; #Simplificando pm fixo
def LimQch(zL,pwL):
    zL=np.array(zL)
    def funcaoqch(zi,pwi): #vazão em m3/s
        return Cc*(zi/100)*np.sqrt(abs(pwi - pm))

    zx = np.arange(zL[0],zL[1],5) # 35 a 65
    px = np.arange(pwL[0],pwL[1],1000)
    X,Y = np.meshgrid(zx, px) # grid of point
    W = funcaoqch(X, Y) # evaluation of the function on the grid
    return [min([valor for linha in W for valor in linha]),max([valor for linha in W for valor in linha])]


LimQch([0,100],[1e5,50e5])



[0.0, 0.03290348005910621]

In [20]:
# Criando simbolica
from casadi import *
nx = 3; nu = 2;
x = MX.sym("x",nx); # Estados
u = MX.sym("u",nu); # Exogena
dudt_max = MX.sym("dudt_max",2); # Exogena
[pbh,pwh,q]=[x[0],x[1],x[2]]
fq = u[0]
zc = u[1]


# Calculo da press�o de intake
F1 = 0.158*((rho*L1*q**2)/(D1*A1**2))*(mu/(rho*D1*q))**(1/4);
F2 = 0.158*((rho*L2*q**2)/(D2*A2**2))*(mu/(rho*D2*q))**(1/4);

funcF1=Function('funcF1',[x],[F1])
funcF2=Function('funcF2',[x],[F2])

xi=np.arange(15/3600,65/3600,0.0001)
#xi=np.arange(1/3600,100/3600,0.0001)
f1=np.zeros_like(xi)
f2=np.zeros_like(xi)
for i in range (0,len(xi)):
    f1[i]=funcF1([0,0,xi[i]])
    f2[i]=funcF2([0,0,xi[i]])
print('Limites F1')
print([min(f1),max(f1)])
print('Limites F2')
print([min(f2),max(f2)])
funcF1([0,0,xi[i]])
plt.plot(xi*3600,f1/1e5,label='F1')
plt.ylabel('bar')
plt.plot(xi*3600,f2/1e5,label='F2')
plt.xlabel('m3/s')
plt.legend()
plt.grid()



Limites F1
[97431.10845445076, 1257145.2528884693]
Limites F2
[233834.66029068184, 3017148.606932326]


In [22]:
350/16


21.875